In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2
import collections
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
import seaborn as sns

Assuming there will be 3 folders: authentic, copy-moved, spliced

In [2]:
# with open('../test_images_modified.pkl','rb') as file:
#     test_authentic, test_copy_images, test_copy_masks, test_spliced_images, test_spliced_masks = pickle.load(file)

In [ ]:
def create_dataset(img_folder):
    img_data_array=[]
    for dir1 in os.listdir(img_folder):
        img_path= os.path.join(img_folder, dir1)
        image= cv2.imread( img_path, cv2.COLOR_BGR2RGB)
        image=np.array(image)
        image = image.astype('float32')
        image /= 255
        img_data_array.append(image)

    return img_data_array

In [ ]:
# extract the image array and class name

holdout_authentic = create_dataset(r'D:\SpoofSense_Assignment\data\holdout\authentic')
holdout_copy_images = create_dataset(r'D:\SpoofSense_Assignment\data\holdout\copy-moved\images')
holdout_copy_masks = create_dataset(r'D:\SpoofSense_Assignment\data\holdout\copy-moved\masks')
holdout_spliced_images = create_dataset(r'D:\SpoofSense_Assignment\data\holdout\spliced\images')
holdout_spliced_masks = create_dataset(r'D:\SpoofSense_Assignment\data\holdout\spliced\masks')

Assuming all the images are equal in number and no need to balance it, we resize the images to 64,64 and feed it to the model

In [ ]:
import cv2

def resize_images(images_list, target_size):
    resized_images = []
    for img in images_list:
        # Determine if the image needs to be enlarged or shrinked
        if img.shape[0] < target_size[0] or img.shape[1] < target_size[1]:
            interpolation = cv2.INTER_LINEAR  # Enlarging
        else:
            interpolation = cv2.INTER_AREA  # Shrinking

        # Resize the image with the appropriate interpolation method
        resized_img = cv2.resize(img, target_size, interpolation=interpolation)
        resized_images.append(resized_img)

    return resized_images

resized_holdout_authentic = resize_images(holdout_authentic, (64,64))
resized_holdout_copy_images = resize_images(holdout_copy_images, (64,64))
# resized_holdout_copy_masks = resize_images(holdout_copy_masks, (64,64))
resized_holdout_spliced_images = resize_images(holdout_spliced_images, (64,64))
# resized_holdout_spliced_masks = resize_images(holdout_spliced_masks, (64,64))

In [ ]:
resized_holdout_forged = resized_holdout_copy_images + resized_holdout_spliced_images

In [ ]:
holdout_labels = [0]*len(resized_holdout_authentic) + [1]*len(resized_holdout_forged)

In [ ]:
holdout_data = resized_holdout_authentic + resized_holdout_forged

In [6]:
from keras.utils import to_categorical

holdout_labels = to_categorical(holdout_labels, num_classes=2)
# y_test = to_categorical(test_labels, num_classes=2)

Run the below cells to load the model:

In [7]:
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Dropout
import keras.backend as K
from keras import optimizers
from keras.layers import LeakyReLU
from keras.applications.resnet50 import ResNet50

In [16]:
resnet_model=ResNet50(weights='imagenet', include_top=False, input_shape=(64,64, 3))

model_aug=Sequential()
model_aug.add(resnet_model)

top_model=Sequential()
top_model.add(Flatten(input_shape=(2, 2, 2048)))

top_model.add(Dense(64, activation='relu'))
model_aug.add(Dropout(0.2))

top_model.add(Dense(1, activation='sigmoid'))

model_aug.add(top_model)

for layer in model_aug.layers[0].layers[:171]:
    layer.trainable=False

model_aug.load_weights('../new_resnet50_weights.h5')
model_aug.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

Run the below cell to get the accuracy of the model.

In [23]:
loss, accuracy = model_aug.evaluate(np.array(holdout_data), np.array(holdout_labels))
print("Accuracy of the model: ",accuracy)

In [24]:
model_preds = []
for i in np.array(holdout_data):
    pred = model_aug.predict(i[np.newaxis, ...])
    if pred < 0.5:
        pred = 0
    elif pred >= 0.5:
        pred = 1
    model_preds.append(pred)

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming model_preds and y_test are numpy arrays or lists containing the predictions and true labels, respectively
conf_matrix = confusion_matrix(holdout_labels, model_preds)

# Create a heatmap using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)

# Set axis labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.show()


In [26]:
# model_aug.load_weights('../new_resnet50_weights.h5')
# model_aug.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])
# loss, accuracy = model_aug.evaluate(np.array(resized_test_data), np.array(test_labels))
# print("Accuracy of the model: ",accuracy)